In [8]:
import pandas as pd
scdata = pd.read_csv('sonoma-county.csv', index_col='Date', parse_dates=True)
scdata = scdata.sort_index()
scdata = scdata.fillna(0)
scdata['Cumulative'] = scdata['Active'] + scdata['Deaths'] + scdata['Recovered']
scdata.tail()

,Active,New,Deaths,Recovered,Cumulative
Date,,,,,
2020-03-23,27.0,8.0,1.0,1.0,29.0
2020-03-24,34.0,2.0,1.0,2.0,37.0
2020-03-25,34.0,2.0,1.0,4.0,39.0
2020-03-26,41.0,8.0,1.0,5.0,47.0
2020-03-27,40.0,7.0,1.0,13.0,54.0


In [9]:
def fit_and_plot(data, ax, color):
    sp_x = np.arange(0, len(data), 1)

    # polyfit returns highest power first
    growth_rate, log_intercept = np.polyfit(sp_x, np.log(data), 1)

    extrapolated_days = np.arange(0, 20, 1)
    extrapolated_values = np.exp(log_intercept) * np.exp(growth_rate * extrapolated_days)
    extrapolated_dates = [data.index[0] + np.timedelta64(1, 'D') * t for t in extrapolated_days]
    # doubling time
    1/growth_rate * np.log(2)
    ax.plot(extrapolated_dates, extrapolated_values, color=color, alpha=0.5)
    ax.plot(data.index, data, linestyle='', marker='o', color=color, alpha=0.3)

In [ ]:
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from matplotlib.dates import MO
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
fig, ax = plt.subplots()
loc = mdates.WeekdayLocator(byweekday=MO)
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

# high growth
first_phase = scdata.loc['2020-03-13':'2020-03-20']['Cumulative']
fit_and_plot(first_phase, ax, 'orange')

# low growth
second_phase = scdata.loc['2020-03-20':]['Cumulative']
fit_and_plot(second_phase, ax, 'green')

# total data trend
ax.plot(scdata.index, scdata['Cumulative'])
ax.set_yscale('log')



# TODO: side by side log and linear
# TODO: create slopes for 2 and 3 day doubling times

plt.show()